Dataset Split

In [4]:
import os
import random
import shutil

# Paths
dataset_dir = "dataset"  # Path to your dataset
output_dir = "DatasetSplit"  # Output directory for train/val split

# Create output directories
train_img_dir = os.path.join(output_dir, "train/images")
val_img_dir = os.path.join(output_dir, "val/images")
train_lbl_dir = os.path.join(output_dir, "train/annotations")
val_lbl_dir = os.path.join(output_dir, "val/annotations")

for dir_path in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(dataset_dir) if f.endswith(".jpg") or f.endswith(".png")]

# Shuffle and split (80% train, 20% val)
random.seed(42)
random.shuffle(image_files)
split_idx = int(0.8 * len(image_files))
train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

# Function to copy files
def copy_files(file_list, src_dir, img_dst_dir, lbl_dst_dir):
    for file_name in file_list:
        # Copy image
        shutil.copy(os.path.join(src_dir, file_name), img_dst_dir)
        # Copy corresponding XML file
        xml_file = file_name.replace(".jpg", ".xml").replace(".png", ".xml")
        shutil.copy(os.path.join(src_dir, xml_file), lbl_dst_dir)

# Copy train and val files
copy_files(train_files, dataset_dir, train_img_dir, train_lbl_dir)
copy_files(val_files, dataset_dir, val_img_dir, val_lbl_dir)

print("Dataset split completed!")


Dataset split completed!


In [3]:
!pip install ultralytics opencv-python matplotlib

Defaulting to user installation because normal site-packages is not writeable


model training

In [5]:
import os
import xml.etree.ElementTree as ET

def convert_voc_to_yolo(voc_dir, output_dir, classes):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for xml_file in os.listdir(voc_dir):
        if not xml_file.endswith(".xml"):
            continue

        # Parse XML
        tree = ET.parse(os.path.join(voc_dir, xml_file))
        root = tree.getroot()

        # Image dimensions
        width = int(root.find("size/width").text)
        height = int(root.find("size/height").text)

        yolo_annotations = []
        for obj in root.findall("object"):
            class_name = obj.find("name").text
            if class_name not in classes:
                continue
            class_id = classes.index(class_name)

            # Get bounding box coordinates
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            bbox_width = (xmax - xmin) / width
            bbox_height = (ymax - ymin) / height

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

        # Save YOLO annotations
        txt_file = os.path.join(output_dir, xml_file.replace(".xml", ".txt"))
        with open(txt_file, "w") as f:
            f.write("\n".join(yolo_annotations))

# Specify paths and classes
train_voc_dir = "DatasetSplit/train/annotations"
val_voc_dir = "DatasetSplit/val/annotations"
train_yolo_dir = "DatasetSplit/train/labels"
val_yolo_dir = "DatasetSplit/val/labels"
classes = ["pothole"]  # Add more classes if needed

# Convert train and val annotations
convert_voc_to_yolo(train_voc_dir, train_yolo_dir, classes)
convert_voc_to_yolo(val_voc_dir, val_yolo_dir, classes)

print("Annotations converted to YOLO format!")


Annotations converted to YOLO format!
